In [1]:
%load_ext autoreload
%autoreload 2
import argparse
from typing import Dict

import torch
import os
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from model.metrics import Metrics
from model.unet import Unet, DEFAULT_UNET_LAYERS
from model.dice_loss import DiceLoss, DiceBCELoss
from datasets.dataset import RetinaSegmentationDataset
from utils.resultPrinter import ResultPrinter
import torch.nn.functional as F

C:\Users\shawn\anaconda3\envs\cs7643-final-rich\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def train_model(model, dataloader, criterion, optimizer, device):
    metrics_tracker = Metrics(device)
    model.train()
    train_running_loss = 0.0
    for ind, (img, lbl) in enumerate(tqdm(dataloader, desc="Training")):
        # Copy to device
        img = img.to(device)
        lbl = lbl.to(device)
        # Make the prediction
        lbl_pred = model(img)
        optimizer.zero_grad()
        # Compute loss
        loss = criterion(lbl_pred, lbl)
        # compute metrics
        metrics_tracker.calculate(lbl_pred, lbl)
        # Running tally
        train_running_loss += loss.item() * img.shape[0]
        # Backward step
        loss.backward()
        optimizer.step()

    # Compute the loss for this epoch
    train_loss = train_running_loss / (ind + 1)
    # Compute the metrics for this epoch
    metrics = metrics_tracker.get_mean_metrics(ind + 1)
    metrics['loss'] = train_loss
    return metrics

In [3]:
def eval_model(model, dataloader, criterion, device):
    metrics_tracker = Metrics(device)
    model.eval()
    eval_running_loss = 0.0
    with torch.no_grad():
        for ind, (img, lbl) in enumerate(tqdm(dataloader, desc="Validation")):
            # Copy to device
            img = img.to(device)
            lbl = lbl.to(device)
            # Make the prediction
            lbl_pred = model(img)
            # Compute loss
            loss = criterion(lbl_pred, lbl)
            # compute metrics
            metrics_tracker.calculate(lbl_pred, lbl)
            # Running tally
            eval_running_loss += loss.item() * img.shape[0]

    # Compute the loss for this epoch
    eval_loss = eval_running_loss / (ind + 1)
    # Compute the metrics for this epoch
    metrics = metrics_tracker.get_mean_metrics(ind + 1)
    metrics['loss'] = eval_loss
    return metrics

In [5]:
rootdir: str = "C:/Users/shawn/Desktop/Development/CS7643/data/DATA_4D_Patches/DATA_4D_Patches"
workers: int = 8
load_encoder_weights: str = None
load_bt_checkpoint: str = None
anneal_tmax: int = 10
anneal_eta: int = 0
run_name: str = "test-drive"
checkpoint_dir: str = "C:/Users/shawn/Desktop/Development/CS7643/checkpoint/"

args = {
    "learning_rate": 0.01,
    "unet_layers": "64-128-256",
    "epochs": 10,
    "batch_size": 64,
    "scheduler": "CosineAnnealing",
    "loss_function": "DiceLoss",
    "dropout": 0.2
}

# Get the device
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"

# Determine the layer sizes of the U-Net
unet_layers = DEFAULT_UNET_LAYERS
if args["unet_layers"]:
    unet_layers = [int(x) for x in args["unet_layers"].split("-")]

# Initialize the model on the GPU
model = Unet(dropout=args["dropout"], hidden_channels=unet_layers).to(device)
if load_encoder_weights:
    model.encoder.load_state_dict(torch.load(load_encoder_weights))
elif load_bt_checkpoint:
    model.encoder.load_state_dict(torch.load(load_bt_checkpoint)["encoder"])
optimizer = torch.optim.Adam(model.parameters(), lr=args["learning_rate"])

# Define scheduler (if necessary)
scheduler = None
if args["scheduler"] == 'CosineAnnealing':
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, anneal_tmax, anneal_eta)
elif args["scheduler"] == 'ReduceOnPlateau':
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

# Select the Loss function
loss_functions = {
    "BCEWithLogitsLoss": torch.nn.BCEWithLogitsLoss(),
    "CrossEntropyLoss": torch.nn.CrossEntropyLoss(),
    "DiceLoss": DiceLoss(),
    "DiceBCELoss": DiceBCELoss()
}
criterion = loss_functions[args["loss_function"]]

# Load the training datasets
training_path = os.path.join(rootdir, "Training")
training_file_basenames = os.listdir(os.path.join(training_path, "images"))
training_dataset = RetinaSegmentationDataset(training_path, training_file_basenames)
training_dataloader = torch.utils.data.DataLoader(
    training_dataset, batch_size=args["batch_size"], num_workers=workers,
    pin_memory=True, shuffle=True)

# Load the validation datasets
validation_path = os.path.join(rootdir, "Validation")
validation_file_basenames = os.listdir(os.path.join(validation_path, "images"))
validation_dataset = RetinaSegmentationDataset(validation_path, validation_file_basenames)
validation_dataloader = torch.utils.data.DataLoader(
    validation_dataset, batch_size=args["batch_size"], num_workers=workers,
    pin_memory=True, shuffle=False)

# Load the validation datasets
test_path = os.path.join(rootdir, "Testing")
test_file_basenames = os.listdir(os.path.join(test_path, "images"))
test_dataset = RetinaSegmentationDataset(test_path, test_file_basenames)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, num_workers=1,
    pin_memory=True, shuffle=False)

# Train / Val loop
training_losses = []
validation_losses = []

# Create a descriptive name for the checkpoints
temp_dict = dict(args)
descrip_name = ""
for key in temp_dict.keys():
    if (key != "rootdir" and
            "load" not in key and
            "checkpoint" not in key and
            "workers" not in key and
            "save_freq" not in key):
        descrip_name += "--" + key + "=" + str(temp_dict[key])
descrip_name = descrip_name.replace(' ', '_').replace('[', '').replace(']', '').replace('\'', '')

# runs dict should be passed to each instance of a results printer. It is only appended to so should be thread safe.
runs: Dict[str, Dict[str, float]] = {}
# create a new results printer for each param setting tested
result_printer = ResultPrinter(descrip_name, runs, run_name=run_name)

epoch_pbar = tqdm(total=args["epochs"], desc="Epochs")

prev_validation_loss = None

Epochs:   0%|                                                                                                                                     | 0/10 [00:00<?, ?it/s]

In [6]:
for i in range(args["epochs"]):

    train_metrics = train_model(model, training_dataloader, criterion, optimizer, device)
    result_printer.print(f'Training metrics: {str(train_metrics)}')
    train_loss = train_metrics['loss']

    validation_metrics = eval_model(model, validation_dataloader, criterion, device)
    result_printer.print(f'Validation metrics: {str(validation_metrics)}')
    validation_loss = validation_metrics['loss']

    result_printer.rankAndSave(validation_metrics)

    training_losses.append(train_loss)
    validation_losses.append(validation_loss)
    epoch_pbar.write("=" * 80)
    epoch_pbar.write("Epoch: {}".format(i))
    epoch_pbar.write("Train Loss : {:.4f}".format(train_loss))
    epoch_pbar.write("Validation Loss : {:.4f}".format(validation_loss))
    epoch_pbar.write("=" * 80)
    epoch_pbar.update(1)

    # Save plot of Train/Validation Loss Per Epoch
    result_printer.makePlots(training_losses, validation_losses, i)

    # Take appropriate scheduler step (if necessary)
    if args["scheduler"] == 'CosineAnnealing':
        scheduler.step()
    elif args["scheduler"] == 'ReduceOnPlateau':
        scheduler.step(validation_loss)

    if prev_validation_loss is not None:
        if abs(prev_validation_loss - validation_loss) / prev_validation_loss < 0.01:
            break

    # if i % args.save_freq == 0:
    # save the model
    state = dict(epoch=i + 1,
                 model=model.state_dict(),
                 optimizer=optimizer.state_dict(),
                 unet_layer_sizes=unet_layers,
                 args=temp_dict)
    torch.save(state, checkpoint_dir + f'unet-best-1208-{descrip_name}-epoch-{i}.pth')

    prev_validation_loss = validation_loss


Training:   0%|                                                                                                                                  | 0/180 [00:00<?, ?it/s]C:\Users\shawn\anaconda3\envs\cs7643-final-rich\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")

Training:  51%|█████████████████████████████████████████████████████████████▏                                                           | 91/180 [00:26<00:15,  5.62it/s]


Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 180/180 [00:42<00:00,  4.22it/s]


Training metrics: {'f1_score': 0.537265965094169, 'sensitivity': 0.6199684855838616, 'specificity': 0.8661241778896914, 'accuracy': 0.8355407343970405, 'auc_roc': 0.8290151261621052, 'mean_iou': 0.38118391945544217, 'ssim': 0.06557724135410455, 'loss': 29.785734346177843}



Epochs:  10%|████████████▌                                                                                                                | 1/10 [00:57<08:36, 57.40s/it]

Validation metrics: {'f1_score': 0.5052895200252533, 'sensitivity': 0.433935759862264, 'specificity': 0.9684198800722758, 'accuracy': 0.9020887247721354, 'auc_roc': 0.8584955024719239, 'mean_iou': 0.3452802722652753, 'ssim': 0.026333794235057818, 'loss': 31.776187235514325}
Epoch: 0
Train Loss : 29.7857
Validation Loss : 31.7762
training loss per epoch: [29.785734346177843]
validation loss per epoch: [31.776187235514325]


...


Training:  52%|██████████████████████████████████████████████████████████████▌                                                          | 93/180 [00:24<00:15,  5.64it/s]


Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 180/180 [00:39<00:00,  4.52it/s]


Training metrics: {'f1_score': 0.8201951172616747, 'sensitivity': 0.816516426205635, 'specificity': 0.9753460698657566, 'accuracy': 0.9554017861684163, 'auc_roc': 0.9359944621721904, 'mean_iou': 0.6952499293618732, 'ssim': 0.004105333435452647, 'loss': 11.526486290825737}



Epochs:  80%|████████████████████████████████████████████████████████████████████████████████████████████████████                         | 8/10 [07:12<01:47, 53.91s/it]

Validation metrics: {'f1_score': 0.7664717721939087, 'sensitivity': 0.7136205937465032, 'specificity': 0.9839075096448262, 'accuracy': 0.9485255432128906, 'auc_roc': 0.8942509396870931, 'mean_iou': 0.6280761444568634, 'ssim': 0.002221194242980952, 'loss': 14.970980173746744}
Epoch: 7
Train Loss : 11.5265
Validation Loss : 14.9710
training loss per epoch: [29.785734346177843, 17.45008659362793, 15.192716725667317, 13.902807299296061, 13.084696430630155, 12.437855381435819, 11.961821746826171, 11.526486290825737]
validation loss per epoch: [31.776187235514325, 21.066141408284505, 23.455560251871745, 16.552221374511717, 16.375254974365234, 15.573962809244792, 14.981948852539062, 14.970980173746744]


In [141]:
from datasets.dataset import RetinaSegmentationDataset
import torchvision.transforms as transforms
from img_transform.transforms import EyeMaskCustomTransform, EyeDatasetCustomTransform


DRIVE_TRANSFORMS = transforms.Compose([
    transforms.ToTensor(),
    torch.nn.ConstantPad2d((0, 75, 0, 56), 0),
    EyeDatasetCustomTransform(mask_threshold=0.25),
])


# Load the validation datasets
test_path = os.path.join(rootdir, "Testing")
test_file_basenames = os.listdir(os.path.join(test_path, "images"))
test_dataset = RetinaSegmentationDataset(test_path, test_file_basenames, has_labels=False, img_transforms=DRIVE_TRANSFORMS)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, num_workers=1,
    pin_memory=True, shuffle=False)

In [172]:
def predict_model(model, dataloader, device):
    model.eval()
    with torch.no_grad():
        for ind, (img, lbl) in enumerate(tqdm(dataloader, desc="Testing")):
            # Copy to device
            img = img.to(device)
            # Make the prediction
            lbl_pred = model(img)
            im = np.round(F.sigmoid(lbl_pred).squeeze(0).squeeze(0).cpu().detach().numpy()[:584, :565])
            im = Image.fromarray(im.astype(np.uint8) * 255)
            im.save(f"C:/Users/shawn/Desktop/Development/CS7643/drive_predicted/{ind}.png")

In [173]:
predict_model(model, test_dataloader, device)

result_printer.close()


Testing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 10.66it/s]


In [71]:
import numpy as np
test = torch.Tensor(np.ones((3, 3)))

In [174]:
import pickle

with open(os.path.join(test_path, "images", test_file_basenames[0]), "rb") as f:
    test_img = pickle.load(f)

array([[[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        ...,
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        ...,
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        ...,
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       ...,

       [[ 8,  6,  7,  0],
        [ 8,  6,  7,  0],
        [ 8,  6,  7,  0],
        ...,
        [ 8,  8, 10,  0],
        [ 8,  8, 10,  0],
        [ 7,  7,  7,  0]],

       [[ 8,  6,  7,  0],
        [ 8,  6,  7,  0],
        [ 8,  6,  7,  0],
        ...,
        [ 7,  7,  9,  0],
        [ 6,  6,  8,  0],
        [ 6,  6,  6,  0]],

       [[ 9,  7,  8,  0],
        [ 9,  7,  8,  0],
        [ 9,  7,  8,  0],
        ...,
        [ 1,  0,  5,  0],
        [ 0,  0

In [108]:
565-640

-75